# WELCOME TO THE TRUMP TWEET GENERATOR AND CLASSIFICATOR

# LOAD LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import os.path
from os import path

import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
#nltk.download('punkt')
#nltk.download('omw-1.4')
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from fastai.text.all import *
import fastai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

import emoji
import re
import string
from collections import Counter
from stop_words import get_stop_words

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print("success")

success


# TWEET DATA CLEANING AND PREPARATION CLASS

In [2]:
class TweetClass():
    
    def __init__(self, path: str) -> None:
        self.raw_data = pd.DataFrame()
        self.clean_data_light = pd.DataFrame()
        self.clean_data_strong = pd.DataFrame()
        self.read_data(path)
        
        #Stemmer and Lemmer
        nltk.LancasterStemmer
        self.ls = LancasterStemmer()
        self.lem = WordNetLemmatizer()
        
    def read_data(self,path: str) -> None:
        self.raw_data = pd.read_csv(path)
        
        pass
        
    def raw_df(self) -> pd.DataFrame():
        return self.raw_data
    
    @staticmethod
    def _additional_space_removal(text: str) -> str:
        text = text.strip()
        text = text.split()
        return " ".join(text)
    
    def _stemming_lemming(self, text: str) -> str:
        words = word_tokenize(text)
        #Stemm
        words_stem = [self.ls.stem(w) for w in words]
        #Lemmatize
        words_lem = [self.lem.lemmatize(w) for w in words_stem]
        return words_lem
    
    @staticmethod
    def _clear_emoji(text: str) -> str:
        for emot in emoji.UNICODE_EMOJI:
            text = re.sub(r'('+emot+')', "_".join(emoji.UNICODE_EMOJI[emot].replace(",","").replace(":","").split()), text)
        return text
    
    @staticmethod  
    def _clear_text_full(text: str) -> str:
        """Lower case and remove quotes, links, html, line brackets, numbers and apostrophes.

        Args:
            text (str): raw text

        Returns:
            str: processed text
        """
        text = str(text).lower()
        text = re.sub(r'https?://\S+|www\.\S+|pic\.twitter\.com/\S+', '', text)
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = re.sub('\'','', text)
        
        return text
    
    @staticmethod  
    def _clear_text_light(text: str) -> str:
        """Remove links and HTML content.

        Args:
            text (str): raw text

        Returns:
            str: processed text
        """
        text = re.sub(r'https?://\S+|www\.\S+|pic\.twitter\.com/\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        
        return text
    
    @staticmethod
    def _remove_stopwords(text: str) -> str:
        stop_words = get_stop_words("en")
        stopwords_dict = Counter(stop_words)
        text = ' '.join([word for word in text.split() if word not in stopwords_dict])
        return text
    
    @staticmethod
    def _tokenize(text: str) -> str:
        words = word_tokenize(text)
        return words
    
    @staticmethod
    def _clean_regex(df: pd.DataFrame) -> pd.DataFrame:
        df['content'] = df['content'].map(lambda x: re.sub(r'\W+', ' ', x))
        df['content'] = df['content'].replace(r'\W+', ' ', regex=True)
        return df
    
    def _light_clean(self) -> pd.DataFrame:
        data = self.raw_data.copy()
        
        try: # column doesn't exist
            data = data.drop(['link','retweets','favorites','mentions','hashtags'], axis=1)
        except:
            pass
        
        data["content"] = data["content"].apply(lambda x: self._clear_text_light(x))
        
        #drop empty tweets
        data = data[data['content'].str.len() > 0]
        data["content"] = data["content"].apply(lambda x: self._tokenize(x))
        
        return data
    
    def _strong_clean(self) -> pd.DataFrame:
        data = self.raw_data.copy()
        
        try: # column doesn't exist
            data = data.drop(['link','retweets','favorites','mentions','hashtags'], axis=1)
        except:
            pass
        
        data["content"] = data["content"].apply(lambda x: self._clear_text_full(x))
        data = self._clean_regex(data)
        data["content"] = data["content"].apply(lambda x: self._remove_stopwords(x))
        data["content"] = data["content"].apply(lambda x: self._stemming_lemming(x))
        
        #drop empty tweets
        data = data[data['content'].str.len() > 0]
        
        return data
    
    def clean_data(self) -> None:
        self.clean_data_light = self._light_clean()
        self.clean_data_strong = self._strong_clean()
    
    def visualize_tweet_content(self):
        self.content_length_histogram(self.raw_data, "Raw Tweets")
        self.content_length_histogram(self.clean_data_light, "Light Cleaned Tweets")
        self.content_length_histogram(self.clean_data_strong, "Strong Cleaned Tweets")
        
        #self.most_common_words_vizualization(self.raw_data, "Raw Tweets")
        self.most_common_words_vizualization(self.clean_data_light, "Light Cleaned Tweets")
        self.most_common_words_vizualization(self.clean_data_strong, "Strong Cleaned Tweets")

    # plot functions
    @staticmethod
    def content_length_histogram(df: pd.DataFrame, title:str = "") -> None:
        content_lengths = df['content'].str.len()

        fig, ax = plt.subplots()
        ax.hist(content_lengths, bins=50)

        ax.set_title('Distribution of content length')
        ax.set_xlabel('Content length')
        ax.set_ylabel('Frequency')
        ax.set_title(title)

        plt.show(block = False)
        
    @staticmethod
    def most_common_words_vizualization(df: pd.DataFrame, title:str = "")->None:
        top = Counter([item for sublist in df['content'] for item in sublist])
        df_reduced = pd.DataFrame(top.most_common(20))
        df_reduced.columns = ['Common_words','count']
        df_reduced.style.background_gradient(cmap='Blues')
        
        fig = px.bar(df_reduced, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
        fig.show()

    


# TWEET GENERATOR CLASS

In [3]:
class TweetGeneratorClass():
    
    def __init__(self) -> None:
        self.train_data = fastai.data.core.DataLoaders()
        self.all_tweets = pd.DataFrame()
        self.created_data = pd.DataFrame()
        self.tweet_column_name = str

    def add_data(self, tweets: pd.DataFrame(), tweet_column_name: str, bs: int=64, seq_len: int=72, n_workers: int=4, save_to_csv=True) -> fastai.text.all.DataBlock:
        """Add data to the class

        Args:
            tweets (pd.DataFrame()): Pandas dataframe
            tweet_column_name (str): Column name in the dataframe, which contains the tweets
            bs: (int): batch size
            seq_len: (int): sequence length of sentences to learn from
            n_workers: (int): number of processes
            save_to_csv=True
 
        Returns:
            train_data (fastai.text.all.DataBlock): Train Data in fastai datablock format
            self.all_tweets (pd.DataFrame()): Real Tweets cleaned including real label (=1) in Pandas Dataframe Format
        """
        
        self.tweet_column_name = tweet_column_name
        
        self.all_tweets[self.tweet_column_name] = tweets[self.tweet_column_name]
        if isinstance(self.all_tweets[self.tweet_column_name][0], list):
            self.all_tweets[self.tweet_column_name] = [' '.join(sent) for sent in self.all_tweets[self.tweet_column_name]]
        
        self.all_tweets["label"] = 1 #label (=1 --> real)

        if save_to_csv:
            self.all_tweets[[self.tweet_column_name, "label"]].to_csv("real_Trump_tweets.csv")
            
        dblock = DataBlock(
           blocks=TextBlock.from_df(self.tweet_column_name, seq_len=seq_len, is_lm=True),
           get_x=ColReader('text'))
        self.train_data = dblock.dataloaders(self.all_tweets, bs=bs, n_workers=4)
        #self.train_data.show_batch(max_n=3)
        
        return self.train_data, self.all_tweets

    def create_model(self, data:fastai.text.all.DataBlock):
        """Create Model for Training

        Args:
            data (data:fastai.text.all.DataBlock): fastai datablock

        Returns:
            None
        """
        # Create deep learning model
        self.learner = language_model_learner(data, AWD_LSTM, path = 'Model/', model_dir='tweet_model/')

    def train_model(self, find_lr: bool=True, lr_iters: int=100, const_lr=None, epochs: int=10):
        """Train model

        Args:
            find_lr (bool): Wanna find a godd lr first
            lr_iters (int): Number of iterations to find best learning rate
            const_lr (float): if None, lr finder is used, else use this lr in combination with set epochs for training
            epochs (int): Number of training loops (epochs)

        Returns:
            None
        """
        
        # Use Learning Rate finder algorithm
        if find_lr and const_lr == None:
            # find the appropriate learning rate
            lr = self.learner.lr_find(num_it=lr_iters)
            print("optimal learning rate: ", lr)
            # find the point where the slope is steepest
            self.learner.recorder.plot_lr_find()

            # Fit the model based on selected learning rate
            self.learner.fit_one_cycle(1, 1e-2)
            
        # OR Use standard learning
        else:
            self.learner.fit(epochs, lr=const_lr)
            self.learner.recorder.plot_loss()
            
        # save model to ./Model/tweet_model/
        self.learner.save("trump_tweeter")
    
    def create_tweet(self, begin_sentence: str="random",  sentence_length: int=20) -> str:
        """Generate One Tweet

        Args:
            begin_sentence (str): string each sentence needs to begin with, if 'random': random first 4 words of random original tweet is used.
            sentence_length (int): length of generated sentence, length=number of tokens, not necessarily words only.

        Returns:
            tweet_txt (str): One Tweet
        """
               
        # load model from ./Model/tweet_model/
        self.learner.load("trump_tweeter")

        # Predict Tweets starting from the given words 
        if begin_sentence == "random":
            rand_tweet = self.all_tweets[self.tweet_column_name ][np.random.randint(low=0, high=self.all_tweets.shape[0])]
            if isinstance(rand_tweet, list):
                rand_begin_sentence = " ".join(rand_tweet[0:4])
            else:
                rand_begin_sentence = " ".join(rand_tweet.split(" ")[0:4]) 
            print(rand_begin_sentence)
            tweet_txt = self.learner.predict(rand_begin_sentence, sentence_length, no_unk=True, temperature=0.75)
        else:
            tweet_txt = self.learner.predict(begin_sentence, sentence_length, temperature=0.75)
        return tweet_txt
    
    def create_tweet_set(self, num_of_tweets:int = 100, begin_sentence: str="random", sentence_length: int=20, save_to_csv=True) -> pd.DataFrame():
        """Generate One Tweet

        Args:
            begin_sentence (str): string each sentence needs to begin with, if 'random': random first 4 words of random original tweet is used.
            sentence_length (int): length of generated sentence, length=number of tokens, not necessarily words only.

        Returns:
            self.created_data (pd.DataFrame()): Pandas Dataframe of as many tweets as generated, with 2 columns: "self.tweet_column_name", label (=0 --> fake)
        """
          
        tweet_set = []
        for idx in range(num_of_tweets):
            if begin_sentence == "random":
                rand_tweet = self.all_tweets[self.tweet_column_name ][np.random.randint(low=0, high=self.all_tweets.shape[0])]
                if isinstance(rand_tweet, list):
                    rand_begin_sentence = " ".join(rand_tweet[0:4])
                else:
                    rand_begin_sentence = " ".join(rand_tweet.split(" ")[0:4]) 
                tweet_set.append(self.learner.predict(rand_begin_sentence, sentence_length, no_unk=True, temperature=0.75))
            else:
                tweet_set.append(self.learner.predict(begin_sentence, sentence_length, temperature=1.0))
                
        self.created_data = pd.DataFrame(tweet_set, columns = [self.tweet_column_name])
        self.created_data["label"] = 0
        
        if save_to_csv:
            self.created_data.to_csv("fake_Trump_tweets.csv")
        
        return self.created_data


In [8]:
# Wanna Train?
Training = False
#Train on full dataset? Switch to false for generator to be faster, or for testing/debugging
train_full = False
##################################################################################################################################################
# DATA CLEANING AND PREPARATION

root_path = os.getcwd()
data_path = f"{root_path}/Data/NLP"

#trumptweets - pre 2018(?)
#realdonaldtrump - post 2018(?)
if train_full:
    tweets_csv = "realdonaldtrump.csv"
else:
    tweets_csv = "realdonaldtrump_small_testset.csv" # only 1000 tweets for testing/debugging

data_path = f"{data_path}/{tweets_csv}"
tweet_col = "content"
trump_tweets = TweetClass(data_path)
trump_tweets.clean_data()
data = trump_tweets._light_clean()

#data = trump_tweets._strong_clean()
#data = trump_tweets.raw_df() # Raw Sentences (no cleaning)
#trump_tweets.visualize_tweet_content()

##################################################################################################################################################
# GENERATOR
##########################################
# INITIALIZE DATA FOR TRAINING

# Init Creator Class
train_trump_tweets = TweetGeneratorClass()

# Load Raw Data from dataframe (tweet column name must be = "content") !NEEDED ALSO FOR PREDICTION!
datablock_tweets, real_tweets = train_trump_tweets.add_data(data, tweet_column_name = tweet_col, bs=32, seq_len=72, n_workers=4, save_to_csv=True)
#print(real_tweets)

# Create ML Model (with hard-coded predifined AWD_LSTM architecture) !NEEDED ALSO FOR PREDICTION!
model = train_trump_tweets.create_model(data=datablock_tweets)

##########################################
# TRAINING

# Wanna Train? Wanna find good learning_rate first? If const_lr is set,NO LR Finding is happening! How many epochs?)
if Training:
    train = train_trump_tweets.train_model(find_lr=True, lr_iters = 100, const_lr=0.005, epochs=10)

##########################################
# CREATE

# Create ONE Tweet! How long should it be?
#tweet = train_trump_tweets.create_tweet(begin_sentence="random", sentence_length=30)
#print(tweet)

# Create MULTIPLE Tweets! How long should it be? CSV Saved to main dir?
fake_tweets = train_trump_tweets.create_tweet_set(num_of_tweets=1000, begin_sentence="random", sentence_length=30, save_to_csv=True)
print("\nGENERATED TWEETS:\n",fake_tweets[tweet_col])



Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


KeyboardInterrupt: 

<h1>CLASSIFIER</h1>

In [14]:
root_path = os.getcwd()

fake_tweets = TweetClass(f"{root_path}/Model/10000_fake_Trump_tweets.csv")
real_tweets = TweetClass(f"{root_path}/Model/10000_real_Trump_tweets.csv")

fake_tweets.clean_data()
real_tweets.clean_data()

print("done")

done


In [19]:
real_tweets_content_list = real_tweets.clean_data_strong['content'].tolist()
fake_tweets_content_list = fake_tweets.clean_data_strong['content'].tolist()

# flat lists
flat_list_real = list(np.concatenate(real_tweets_content_list).flat)
flat_list_fake = list(np.concatenate(fake_tweets_content_list).flat)

# Create bag-of-word features of tweets with vectorizer
vectorizer = CountVectorizer()

real_bow = vectorizer.fit_transform(flat_list_real)
fake_bow = vectorizer.fit_transform(flat_list_fake)

# Use TruncatedSVD to reduce dimensionality of feature
svd = TruncatedSVD(n_components=310)  
real_bow = svd.fit_transform(real_bow)
fake_bow = svd.fit_transform(fake_bow)

print("done")

done


In [20]:
# Concatenate bag of word vectors of real and fake tweets
X = np.concatenate((real_bow, fake_bow), axis=0)
y = np.concatenate((np.ones(len(real_bow)), np.zeros(len(fake_bow))), axis=0)

# Shuffle data randomly
X, y = shuffle(X, y, random_state=0)

clf = LogisticRegression(random_state=0)

scores = cross_val_score(clf, X, y, cv=5)
print('Cross validation scores:', scores)
print('Mean score:', scores.mean())

Cross validation scores: [0.5938249  0.5997342  0.59923583 0.59909343 0.5978831 ]
Mean score: 0.5979542919524408


In [ ]:
# Concatenate bag of word vectors of real and fake tweets
X = np.concatenate((real_bow, fake_bow), axis=0)
y = np.concatenate((np.ones(len(real_bow)), np.zeros(len(fake_bow))), axis=0)

# Shuffle data randomly
X, y = shuffle(X, y, random_state=0)

svm = SVC(kernel='linear')
scores = cross_val_score(svm, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())
print("jo")